### Knowledge Graph From Text Using llama-3.1-70b-versatile

In [1]:
import os
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [3]:
# Set your Groq API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(GROQ_API_KEY)

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


In [32]:
class Entity(BaseModel):
    name: str = Field(description="Name of the entity")
    type: str = Field(description="Type of the entity (e.g., PersonalityTrait, Ability, Interest, Skill, Knowledge etc.)")


In [33]:
class Relationship(BaseModel):
    subject: str = Field(description="Subject entity of the relationship")
    predicate: str = Field(description="Type of relationship")
    object: str = Field(description="Object entity of the relationship")

In [34]:
class KnowledgeGraphOutput(BaseModel):
    entities: List[Entity] = Field(description="List of entities extracted from the text")
    relationships: List[Relationship] = Field(description="List of relationships extracted from the text")

In [35]:
# Create the output parser
output_parser = PydanticOutputParser(pydantic_object=KnowledgeGraphOutput)

In [36]:
# Define the prompt template
prompt_template = PromptTemplate(
    template="""
    Extract entities and relationships from the following text to construct a knowledge graph:

    {text}

    {format_instructions}

    Provide the output in the specified JSON format.
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [46]:
# Initialize the Groq LLM
llm = ChatGroq(temperature=0, model_name="llama-3.1-70b-versatile")

In [47]:
# Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [48]:
def extract_kg_elements(text):
    # Run the chain
    result = chain.run(text=text)
    
    # Parse the output
    parsed_output = output_parser.parse(result)
    
    return parsed_output.entities, parsed_output.relationships

In [49]:
sample_text = """
    Investigative people are inquisitive, analytical and like to learn. Investigative professions require one to carefully analyse the tasks at hand. Many investigative professionals are an expert in their particular field and solve problems through research and analysis. Typical hobbies of investigative people include intellectual sports such as Sudoku and chess, learning different languages or following additional courses just for the fun of it. One will encounter many investigative professions in academic education, IT, healthcare and innovative companies. Keywords: analytical, inquisitive, gaining knowledge, continuous learning, specialist.
"""

In [50]:
entities, relationships = extract_kg_elements(sample_text)

In [39]:
print("Entities:")
for entity in entities:
    print(f"- {entity.name} ({entity.type})")

Entities:
- Investigative people (Person)
- Investigative professions (Profession)
- Academic education (Organization)
- IT (Industry)
- Healthcare (Industry)
- Innovative companies (Organization)
- Sudoku (Hobby)
- Chess (Hobby)
- Learning different languages (Hobby)
- Following additional courses (Hobby)
- Analytical (Trait)
- Inquisitive (Trait)
- Gaining knowledge (Goal)
- Continuous learning (Goal)
- Specialist (Role)


In [51]:
print("\nRelationships:")
for rel in relationships:
    print(f"- {rel.subject} {rel.predicate} {rel.object}")


Relationships:
- Investigative people are Analytical
- Investigative people are Inquisitive
- Investigative professions require Analysis
- Investigative people enjoy Sudoku
- Investigative people enjoy Chess
- Investigative people enjoy Learning different languages
- Investigative people enjoy Following additional courses
- Investigative professions are found in Academic education
- Investigative professions are found in IT
- Investigative professions are found in Healthcare
- Investigative professions are found in Innovative companies
- Investigative people use Research
- Investigative people use Analysis
- Investigative people use Problem-solving
